In [1]:
%load_ext autoreload
%autoreload 2
#!/usr/bin/env python
# -*- coding: utf-8 -*-
# @Time    : 2022/6/9 15:46
# @Author  : Wang Yujia
# @File    : basicinfo_datawithnp.py
# @Description : 主要对于处理后（指计算出n与p并且经过threash）的data，进行一些分析

# 1. Preparations
## 1.1 全局设置

In [3]:
outcomes_orignal_path = "../data/outcomes.tsv"
data_path = "../data/datawithnp_2_selected.csv"

import pandas as pd
import numpy as np
import os
from visdom import Visdom

unique_setting = ['id','bidincrement','bidfee','retail']

## 1.2 Read in
1. 读入target data
    - 一个是全部的target data
    - 一个是training用的，按照auctions的不同分开的target data

In [4]:
data = pd.read_csv(data_path,encoding="utf-8")
outcomes = pd.read_csv(outcomes_orignal_path,sep='\t')
data.head(10),data.shape

(   auction_id  product_id                                      item  \
 0       86827    10009602  sony-ericsson-s500i-unlocked-mysterious-   
 1       87964    10009881            psp-slim-lite-sony-piano-black   
 2       87965    10009881            psp-slim-lite-sony-piano-black   
 3       88638    10006115  sony-ericsson-s500i-unlocked-mysterious-   
 4       88639    10006115  sony-ericsson-s500i-unlocked-mysterious-   
 5       88693    10008975  sony-ericsson-s500i-unlocked-mysterious-   
 6       88694    10008975  sony-ericsson-s500i-unlocked-mysterious-   
 7       89434    10007148  ipod-touch-apple-8gb-with-software-upgra   
 8       89435    10007148  ipod-touch-apple-8gb-with-software-upgra   
 9       90127    10007148  ipod-touch-apple-8gb-with-software-upgra   
 
                                             desc  retail   price  finalprice  \
 0  Sony Ericsson S500i Unlocked Mysterious Green  499.99   13.35       13.35   
 1               PSP Slim & Lite Sony Piano 

In [5]:
# target data
target_path = r"../data/targets"
# target_path里有全部的target data地址
target_all_path = os.listdir(target_path)
len_target = len(target_all_path)
print(f"一共有 *{len_target}* 组 target data")

一共有 *1196* 组 target data


# 2. Info
## 2.1 Do different 'product_id' has different 'item'?
1. 重要结论：**'item'是不全的描述，不应该以此为参考**


In [16]:
data_prod = data[['product_id','item','desc']].copy()
data_prod.drop_duplicates(inplace=True,ignore_index=True)

data_prodid = data[['product_id']].copy()
data_prodid.drop_duplicates(inplace=True,ignore_index=True)

data_item = data[['item']].copy()
data_item.drop_duplicates(inplace=True,ignore_index=True)

print(f"outcomes.tsv中一共记录了 *{data_prod.shape[0]}* 个['product_id','item','desc']对")
print(f"outcomes.tsv中一共记录了 *{data_prodid.shape[0]}* 个['product_id']")
print(f"outcomes.tsv中一共记录了 *{data_item.shape[0]}* 个['item']")

outcomes.tsv中一共记录了 *1122* 个['product_id','item','desc']对
outcomes.tsv中一共记录了 *1117* 个['product_id']
outcomes.tsv中一共记录了 *905* 个['item']


In [17]:
# 问题不大
prod_not_uniq_1=pd.DataFrame(columns=['product_id','item','desc'])
for i in range(0,data_prodid.shape[0]):
    prodid = data_prodid.iloc[i,0]
    prod = data_prod[(data_prod['product_id']==prodid)].copy()
    prod.drop_duplicates(inplace=True,ignore_index=True)
    if(prod.shape[0]!=1):
        prod_not_uniq_1 = prod_not_uniq_1.append(prod,ignore_index=True)

print(f"有 *{prod_not_uniq_1['product_id'].nunique()}* 个'product_id' 对应了多个'item'")

[Errno 11001] getaddrinfo failed
C:\Users\Wang Yujia\AppData\Local\Temp\ipykernel_32020\1566985900.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  prod_not_uniq_1 = prod_not_uniq_1.append(prod,ignore_index=True)


有 *5* 个'product_id' 对应了多个'item'


In [18]:
# 研究一下
prod_not_uniq_2=pd.DataFrame(columns=['product_id','item','desc'])

for i in range(0,data_item.shape[0]):
    item = data_item.iloc[i,0]
    prod = data_prod[data_prod['item']==item].copy()
    prod.drop_duplicates(inplace=True,ignore_index=True)
    if(prod.shape[0]!=1):
        prod_not_uniq_2 = prod_not_uniq_2.append(prod,ignore_index=True)
print(f"有 *{prod_not_uniq_2['item'].nunique()}* 个'item' 对应了多个'product_id'")

item = prod_not_uniq_2['item'].unique()

C:\Users\Wang Yujia\AppData\Local\Temp\ipykernel_32020\1690109597.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  prod_not_uniq_2 = prod_not_uniq_2.append(prod,ignore_index=True)
[Errno 11001] getaddrinfo failed


有 *130* 个'item' 对应了多个'product_id'


从data里看看这130个怎么回事，其实也没什么，输出一组`tmp_csv`看看

In [14]:
data_item = data[ data['item'].isin(item)]

tmp_csv = data_item[data_item['item'] == item[0]].copy()
tmp_csv.reset_index(inplace=True,drop=True)
tmp_csv.to_csv("tmp_csv.csv",header=True,encoding="utf-8",index=False)
tmp_csv

,auction_id,product_id,item,desc,retail,price,finalprice,bidincrement,bidfee,winner,...,endtime_str,flg_click_only,flg_beginnerauction,flg_fixedprice,flg_endprice,id,N,cnt_uniq,cnt_n_2,P
0,86827,10009602,sony-ericsson-s500i-unlocked-mysterious-,Sony Ericsson S500i Unlocked Mysterious Green,499.99,13.35,13.35,0.15,0.75,Racer11,...,19:52 PDT 09-16-2008,0,0,0,0,0,89,69,1,0.014493
1,88638,10006115,sony-ericsson-s500i-unlocked-mysterious-,Sony Ericsson S500i Unlocked Mysterious Green,499.99,19.65,19.65,0.15,0.75,Mokkis,...,22:02 PDT 08-23-2008,0,0,0,0,0,131,69,1,0.014493
2,88639,10006115,sony-ericsson-s500i-unlocked-mysterious-,Sony Ericsson S500i Unlocked Mysterious Green,499.99,47.10,47.10,0.15,0.75,Superloeffel,...,14:23 PDT 08-24-2008,0,0,0,0,0,314,69,1,0.014493
3,88693,10008975,sony-ericsson-s500i-unlocked-mysterious-,Sony Ericsson S500i Unlocked Mysterious Green,499.99,55.20,55.20,0.15,0.75,Danydemir80,...,22:44 PDT 08-22-2008,0,0,0,0,0,368,69,1,0.014493
4,88694,10008975,sony-ericsson-s500i-unlocked-mysterious-,Sony Ericsson S500i Unlocked Mysterious Green,499.99,86.10,86.10,0.15,0.75,Destination8,...,07:10 PDT 08-24-2008,0,0,0,0,0,574,69,1,0.014493
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,99187,10008975,sony-ericsson-s500i-unlocked-mysterious-,Sony Ericsson S500i Unlocked Mysterious Green,499.99,34.80,34.80,0.15,0.75,Jonniboiuk,...,08:56 PDT 10-18-2008,0,0,0,0,0,232,69,1,0.014493
61,102078,10009602,sony-ericsson-s500i-unlocked-mysterious-,Sony Ericsson S500i Unlocked Mysterious Green,499.99,32.10,32.10,0.15,0.75,Blueoscar,...,18:48 PDT 09-10-2008,0,0,0,0,0,214,69,1,0.014493
62,107728,10009642,sony-ericsson-s500i-unlocked-mysterious-,Sony Ericsson S500i Unlocked Mysterious Green,499.99,17.25,17.25,0.15,0.75,Porygon,...,07:46 PDT 10-11-2008,0,0,0,0,0,115,69,1,0.014493
63,115849,10009602,sony-ericsson-s500i-unlocked-mysterious-,Sony Ericsson S500i Unlocked Mysterious Green,499.99,24.90,24.90,0.15,0.75,Catalana71,...,03:18 PDT 11-07-2008,0,1,0,0,0,166,69,1,0.014493


可以验证，`prod_not_uniq_2`和`prod_not_uniq_1` **无重合**，所以可以放心处理两组数据不会产生冲突

## 2.2 Do different 'item' has different 'desc'?
1. 重要结论：**'item'是不全的描述，不应该以此为参考**

In [81]:
data_prod = data[['item','desc']].copy()
data_prod.drop_duplicates(inplace=True,ignore_index=True)

data_item = data[['item']].copy()
data_item.drop_duplicates(inplace=True,ignore_index=True)

data_desc = data[['desc']].copy()
data_desc.drop_duplicates(inplace=True,ignore_index=True)

print(f"outcomes.tsv中一共记录了 *{data_prod.shape[0]}* 个['item','desc']对")
print(f"outcomes.tsv中一共记录了 *{data_item.shape[0]}* 个['item']")
print(f"outcomes.tsv中一共记录了 *{data_desc.shape[0]}* 个['desc']")

outcomes.tsv中一共记录了 *899* 个['item','desc']对
outcomes.tsv中一共记录了 *897* 个['item']
outcomes.tsv中一共记录了 *899* 个['desc']


2. 有2个item对应了多个desc，研究一下

In [82]:
# data_prod = data[['product_id','item','desc']].copy()
# data_prod.drop_duplicates(inplace=True,ignore_index=True)
prod_not_uniq = pd.DataFrame(columns=['product_id','item','desc'])

for i in range(0,data_item.shape[0]):
    item = data_item.iloc[i,0]
    desc = data_prod[data_prod['item']==item].copy()
    desc.drop_duplicates(inplace=True,ignore_index=True)
    if(desc.shape[0]!=1):
        prod_not_uniq = prod_not_uniq.append(desc,ignore_index=True)
print(f"有 *{prod_not_uniq['item'].nunique()}* 个'item' 对应了多个'desc'")
prod_not_uniq

有 *2* 个'item' 对应了多个'desc'


,product_id,item,desc
0,NaN,mario-sonic-at-the-olympic-winter-games-,Mario & Sonic at the Olympic Winter Games (DS)
1,NaN,mario-sonic-at-the-olympic-winter-games-,Mario & Sonic at the Olympic Winter Games (Wii)
2,NaN,star-wars-the-clone-wars-republic-heroes,Star Wars: The Clone Wars - Republic Heroes (Wii)
3,NaN,star-wars-the-clone-wars-republic-heroes,Star Wars: The Clone Wars - Republic Heroes (PC)


## 2.3 Do different 'product_id' has different 'desc'?

In [89]:
data_prod = data[['product_id','desc']].copy()
data_prod.drop_duplicates(inplace=True,ignore_index=True)

data_prodid = data[['product_id']].copy()
data_prodid.drop_duplicates(inplace=True,ignore_index=True)

data_item = data[['desc']].copy()
data_item.drop_duplicates(inplace=True,ignore_index=True)

print(f"outcomes.tsv中一共记录了 *{data_prod.shape[0]}* 个['product_id','desc']对")
print(f"outcomes.tsv中一共记录了 *{data_prodid.shape[0]}* 个['product_id']")
print(f"outcomes.tsv中一共记录了 *{data_item.shape[0]}* 个['desc']")

outcomes.tsv中一共记录了 *1011* 个['product_id','desc']对
outcomes.tsv中一共记录了 *1005* 个['product_id']
outcomes.tsv中一共记录了 *899* 个['desc']


## 2.4 Do same 'desc' have same 'retail'?

In [101]:
data_prod = data[['retail','desc']].copy()
data_prod.drop_duplicates(inplace=True,ignore_index=True)

data_desc = data[['desc']].copy()
data_desc.drop_duplicates(inplace=True,ignore_index=True)

print(f"outcomes.tsv中一共记录了 *{data_prod.shape[0]}* 个['retail','desc']对")
print(f"outcomes.tsv中一共记录了 *{data_desc.shape[0]}* 个['desc']")

outcomes.tsv中一共记录了 *1092* 个['retail','desc']对
outcomes.tsv中一共记录了 *907* 个['desc']


1. find out 有很多desc对应了多个“retail”，这个时候应该把他们当做不同的“auction”！！

In [105]:
prod_not_uniq = pd.DataFrame(columns=['retail','desc'])

for i in range(0,data_desc.shape[0]):
    tmp = data_desc.iloc[i,0]
    desc = data_prod[data_prod['desc']==tmp].copy()
    desc.drop_duplicates(inplace=True,ignore_index=True)
    if(desc.shape[0]!=1):
        prod_not_uniq = prod_not_uniq.append(desc,ignore_index=True)

print(f"有 *{prod_not_uniq['desc'].nunique()}* 个'desc' 对应了多个'retail'")


有 *150* 个'desc' 对应了多个'retail'


desc
1 Ounce Gold Bar (31.10g)                             7
20 Bids Voucher                                       3
300 Bids Voucher                                      3
300 FreeBids Voucher                                  2
50 Bids Voucher                                       3
                                                     ..
Wii Play (includes Wii Remote Control)                2
Wii Sports Resort with Wii MotionPlus                 4
Wii Wireless Sensor Bar                               2
Wii | Nintendo Console + Wii Sports                   4
Xbox 360 Arcade Console with Sega Superstar Tennis    2
Length: 150, dtype: int64

## 2.5 How many uniq auctions here?

In [3]:
data_grouped = data.groupby(unique_setting,as_index=False)
data_g = data_grouped.size()
print(f"一共有 *{data_g.shape[0]}* 个settings")
data_g.describe()

一共有 *1196* 个settings


,id,bidincrement,bidfee,retail,size
count,1196.000000,1196.000000,1196.000000,1196.00000,1196.000000
mean,785.630435,0.126647,0.688420,281.46388,57.127090
std,468.415208,0.054292,0.073821,422.22706,98.325974
min,0.000000,0.010000,0.600000,8.59000,4.000000
25%,390.500000,0.120000,0.600000,44.99000,19.000000
50%,771.000000,0.150000,0.750000,82.89000,31.000000
75%,1134.250000,0.150000,0.750000,313.38000,54.000000
max,1748.000000,0.240000,0.750000,2799.99000,1596.000000


1. 有多少样本数>=100的setting？

In [4]:
print(data_g['size'].map(lambda x: x >= 100).sum())

133


## 2.6 How many records in data/ in avg?
1. 某个auction的duration=10出现了k次，这里会算作k条records

In [8]:
target_lenth = []
# target_path = r"../data/targets_5"
target_path = r"../data/targets_5_DA"
for i in range(len_target):
    target_path_i_path = os.path.join(target_path,target_all_path[i])
    target_df = pd.read_csv(target_path_i_path,encoding="utf-8")

    target_lenth.append(len(target_df))
assert target_lenth.__len__() == len_target,"Wrong"

print("records总数为：",sum(target_lenth))
print("在所有的target data files 中")
print("records的数目最小为：",min(target_lenth))
avg_len = np.mean(target_lenth)
print("records的数目平均为：",avg_len)
print("records的数目中位数为：",np.median(target_lenth))

# 统计长度小于avg_len （K）的target data file数目
cnt = 0
K = 60
for i in range(len_target):
    if(target_lenth[i] < K):
        cnt+=1

print(f"records的数目小于{K}的files数目有{cnt}个，占比{cnt/len_target}")

records总数为： 159591
在所有的target data files 中
records的数目最小为： 23
records的数目平均为： 133.43729096989966
records的数目中位数为： 54.0
records的数目小于60的files数目有674个，占比0.5635451505016722


# 3. Draw
## 3.1 Draw the distrb. of target data
1. 有点bug？


In [19]:
# target data
target_path = r"../data/targets"
# target_path里有全部的target data地址
target_all_path = os.listdir(target_path)
len_target = len(target_all_path)
print(f"一共有 *{len_target}* 组 target data")

一共有 *1196* 组 target data


In [25]:
viz_env_str = "target_distrb"
viz = Visdom(viz_env_str)
print("Done")

[Errno 11001] getaddrinfo failed
Visdom python client failed to establish socket to get messages from the server. This feature is optional and can be disabled by initializing Visdom with `use_incoming_socket=False`, which will prevent waiting for this request to timeout.


Done


Setting up a new session...
[Errno 11001] getaddrinfo failed
[Errno 11001] getaddrinfo failed
[Errno 11001] getaddrinfo failed
[Errno 11001] getaddrinfo failed
[Errno 11001] getaddrinfo failed
[Errno 11001] getaddrinfo failed
[Errno 11001] getaddrinfo failed
[Errno 11001] getaddrinfo failed
[Errno 11001] getaddrinfo failed
[Errno 11001] getaddrinfo failed


Exception in user code:
------------------------------------------------------------


Traceback (most recent call last):
  File "D:\Anaconda\envs\py39\lib\site-packages\urllib3\connection.py", line 174, in _new_conn
    conn = connection.create_connection(
  File "D:\Anaconda\envs\py39\lib\site-packages\urllib3\util\connection.py", line 72, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
  File "D:\Anaconda\envs\py39\lib\socket.py", line 954, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
socket.gaierror: [Errno 11001] getaddrinfo failed

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "D:\Anaconda\envs\py39\lib\site-packages\urllib3\connectionpool.py", line 703, in urlopen
    httplib_response = self._make_request(
  File "D:\Anaconda\envs\py39\lib\site-packages\urllib3\connectionpool.py", line 398, in _make_request
    conn.request(method, url, **httplib_request_kw)
  File "D:\Anaconda\envs\py39\lib\site-packages\ur

Done


In [7]:
for i in range(len_target):
    target_path_i_path = os.path.join(target_path,target_all_path[i])
    target_df = pd.read_csv(target_path_i_path,encoding="utf-8")

    win_str = str(target_df)
    title_str = "Target Distrb. of "+win_str

    viz.line(X = [0.],Y = [0.], env=viz_env_str, win=win_str, opts= dict(title=title_str))
    viz.line(X = target_df.N,Y= target_df.P, env=viz_env_str, win=win_str, opts= dict(title=title_str))

print("Done")

[Errno 11001] getaddrinfo failed
[Errno 11001] getaddrinfo failed


Exception in user code:
------------------------------------------------------------


Traceback (most recent call last):
  File "D:\Anaconda\envs\py39\lib\site-packages\urllib3\connection.py", line 174, in _new_conn
    conn = connection.create_connection(
  File "D:\Anaconda\envs\py39\lib\site-packages\urllib3\util\connection.py", line 72, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
  File "D:\Anaconda\envs\py39\lib\socket.py", line 954, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
socket.gaierror: [Errno 11001] getaddrinfo failed

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "D:\Anaconda\envs\py39\lib\site-packages\urllib3\connectionpool.py", line 703, in urlopen
    httplib_response = self._make_request(
  File "D:\Anaconda\envs\py39\lib\site-packages\urllib3\connectionpool.py", line 398, in _make_request
    conn.request(method, url, **httplib_request_kw)
  File "D:\Anaconda\envs\py39\lib\site-packages\ur

Exception in user code:
------------------------------------------------------------


Traceback (most recent call last):
  File "D:\Anaconda\envs\py39\lib\site-packages\urllib3\connection.py", line 174, in _new_conn
    conn = connection.create_connection(
  File "D:\Anaconda\envs\py39\lib\site-packages\urllib3\util\connection.py", line 72, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
  File "D:\Anaconda\envs\py39\lib\socket.py", line 954, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
socket.gaierror: [Errno 11001] getaddrinfo failed

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "D:\Anaconda\envs\py39\lib\site-packages\urllib3\connectionpool.py", line 703, in urlopen
    httplib_response = self._make_request(
  File "D:\Anaconda\envs\py39\lib\site-packages\urllib3\connectionpool.py", line 398, in _make_request
    conn.request(method, url, **httplib_request_kw)
  File "D:\Anaconda\envs\py39\lib\site-packages\ur

...

Exception in user code:
------------------------------------------------------------


Traceback (most recent call last):
  File "D:\Anaconda\envs\py39\lib\site-packages\urllib3\connection.py", line 174, in _new_conn
    conn = connection.create_connection(
  File "D:\Anaconda\envs\py39\lib\site-packages\urllib3\util\connection.py", line 72, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
  File "D:\Anaconda\envs\py39\lib\socket.py", line 954, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
socket.gaierror: [Errno 11001] getaddrinfo failed

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "D:\Anaconda\envs\py39\lib\site-packages\urllib3\connectionpool.py", line 703, in urlopen
    httplib_response = self._make_request(
  File "D:\Anaconda\envs\py39\lib\site-packages\urllib3\connectionpool.py", line 398, in _make_request
    conn.request(method, url, **httplib_request_kw)
  File "D:\Anaconda\envs\py39\lib\site-packages\ur

Exception in user code:
------------------------------------------------------------


Traceback (most recent call last):
  File "D:\Anaconda\envs\py39\lib\site-packages\urllib3\connection.py", line 174, in _new_conn
    conn = connection.create_connection(
  File "D:\Anaconda\envs\py39\lib\site-packages\urllib3\util\connection.py", line 72, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
  File "D:\Anaconda\envs\py39\lib\socket.py", line 954, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
socket.gaierror: [Errno 11001] getaddrinfo failed

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "D:\Anaconda\envs\py39\lib\site-packages\urllib3\connectionpool.py", line 703, in urlopen
    httplib_response = self._make_request(
  File "D:\Anaconda\envs\py39\lib\site-packages\urllib3\connectionpool.py", line 398, in _make_request
    conn.request(method, url, **httplib_request_kw)
  File "D:\Anaconda\envs\py39\lib\site-packages\ur

KeyboardInterrupt: 